# Poker-Solver Implementation
This notebook is an implementation of a poker solver using counterfactual regret minimization.
Pre-requisites to run
- Jupyter Notebook
- Python 2.7 Environment (32 bit installation)
- PypokerEval - https://sourceforge.net/projects/fpdb/files/fpdb/pypoker-eval-win32/
- pandas, numpy, and other libraries which are imported in the notebook

## Libraries

In [1]:
import os
import pokereval
pe = pokereval.PokerEval()
import scipy.misc
%pylab inline

Populating the interactive namespace from numpy and matplotlib


## Global Variables

In [2]:
deckSize = 52
rankSize = 13
suitSize = 4
numHands = 1326
numP2Hands = 1255
suits = ["h", 'd', 'c', 's']
ranks = ["A","K","Q","J","T","9","8","7","6","5","4","3","2"]
cards = []
for i in suits:
    for j in reversed(ranks):
        cards.append(j + i)

## Utility Functions

In [3]:
#Checks if there are any conflicts in two lists of cards
def hasConflicts(cardsl, cardsr):
    s = set()
    for i in cardsl:
        s.add(i)
    for i in cardsr:
        if i in s:
            return True
    return False

def frequencyHandsWithConflicts(handArray, cards, freq):
    for i in cards:
        if i < deckSize:
            handArray[i,:] = freq
            handArray[:,i] = freq

#Just a special case of sethandswithconflicts where num = 0
def EliminateHandsWithConflicts(handArray, cards):
    frequencyHandsWithConflicts(handArray, cards, 0)

## Objects

### Equity Object
- This object is responsible for storing an array of equities of every hand vs every other hand on a given board.

In [4]:
def handVhandEquity(hand, hand2, board):
    if hasConflicts(hand,hand2) or hasConflicts(hand,board) or hasConflicts(hand2,board):
        return -1
    else:
        game = pe.poker_eval(game = 'holdem', pockets = [hand,hand2], board = board)
        total = game["info"][0]
        wins = game["eval"][0]["winhi"]
        ties = game["eval"][0]['tiehi']
        return (float(wins) + ties/2.0)/float(total)

In [5]:
class EquityObj:
    def __init__(self, b):
        self.board = b
        self.eObj = numpy.zeros((deckSize, deckSize,deckSize,deckSize))
        #If a file already exists for this? do nothing
        if os.path.isfile(self.Filename()):
            self.eObj = numpy.load(self.Filename())
        else:
            #Otherwise save it
            self.saveArray()
    
    #Saves the array as a file
    def saveArray(self):
        for i in range(deckSize):
            for j in range(deckSize):
                for left in range(deckSize):
                    for right in range(deckSize):
                        hand = [i,j]
                        hand2 = [left,right]
                        self.eObj[i][j][left][right] = handVhandEquity(hand,hand2,self.board)
        numpy.save(self.Filename(),self.eObj)
                        
    #Returns the file name of an equity object
    def Filename(self):
        boardString = ""
        boardAsStrings = pe.card2string(self.board)
        for i in boardAsStrings:
            if i != '__':
                boardString = boardString + i
        if boardString == '':
            boardString = 'preflop'
        boardString = boardString + '.ea.npy'
        return boardString

### Range Object

In [6]:
class Range:
    def __init__(self, initPercent = None):
        self.range = numpy.zeros((deckSize,deckSize))
        if (initPercent != None):
            for i in range(deckSize):
                for j in range(i+1, deckSize):
                    self.range[i][j] = initPercent
    
    def printRange(self):
        for i in range(deckSize):
            for j in range(deckSize):
                print(pe.card2string(i),pe.card2string(j),self.range[i][j])
            
    #Will tell you how often hand is in range
    def getFreq(self, hand):
        card1 = hand[0]
        card2 = hand[1]
        if card1 > card2:
            card1,card2 = card2,card1
        return self.range[card1][card2]
    
    #Number of hands
    def getNumHands(self):
        return sum(self.range)
    
    #Takes in a list of cards and outputs the number of cards in a range without conflicts
    def getHandsNoConflicts(self, cardslist):
        t = numpy.copy(self.range)
        self.removeConflicts(t,cardslist)
        return sum(t)
    
    def removeConflicts(self, arr, cards):
        for i in cards:
            if i < deckSize:
                arr[i, :] = 0
                arr[:, i] = 0
    
    def setPercent(self, hand, frequency):
        card1 = hand[0]
        card2 = hand[1]
        if card1 > card2:
            card1,card2 = card2,card1
        self.range[card1][card2] = frequency
    

    def setAllFrequencies(self, frequency):
        for i in range(deckSize):
            for j in range(i + 1, deckSize):
                self.range[i][j] = frequency
    
    def scaleFrequencies(self, scale):
        self.range = self.range * scale
        
    def getHandNonCombo(self, r1, r2, suited):
        handNum = 0.0
        freq = 0.0
        for i in suits:
            for j in suits:
                card1 = r1 + i
                card2 = r2 + j
                if (suited and i !=j) or (not suited and i == j) or card1 == card2:
                    continue
                handNum+=1
                freq+=self.getFreq(pe.string2card([card1,card2]))
        frequencyTotal = freq / handNum
        return frequencyTotal
        
        
    def _repr_svg_(self):
        res = '<svg xmlns = "http://www.w3.org/2000/svg" version = "1.1" width = "260" height = "260">'
        for i in range(rankSize):
            for j in range(rankSize):
                freq = self.getHandNonCombo(ranks[i], ranks[j], i > j)
                #Represent based on darkness proportional to fraction
                hexcolor = '#%02x%02x%02x' % (255 * (1-freq),255*(1-freq),255)
                res += '<rect x="' + str(i * 20) + '" y="' + str(j * 20) + '" width="20" height = "20" fill ="'\
                       + hexcolor + '"></rect>'
                res += '<text x=' + str(i * 20) +' y='+str((j + 1) * 20) + ' font-size = 12 >' + ranks[i]\
                + ranks[j] + '</text>'
        res += '</svg>'
        return res
              

In [7]:
def handVsRange(hand, range_, eObj):
    card1, card2 = hand
    equities = eObj.eObj[card1][card2, :, :]
    RangePlayer = np.copy(range_.range)
    EliminateHandsWithConflicts(RangePlayer, hand + eObj.board)
    return sum(np.multiply(equities, RangePlayer)) / sum(RangePlayer)

### Game Tree
A game tree is represented by two objects, a node and a tree

#### Node Class

In [27]:
class Node:
    def __init__(self, player, sb_stack, bb_stack, parent):
        self.player = player
        self.sb_stack = sb_stack
        self.bb_stack = bb_stack
        self.parent = parent
    #Player: cane either be sb or bb
    def getStack(self, player):
        if (player == "SB"):
            return self.sb_stack
        elif (player == "BB"):
            return self.bb_stack
        else:
            print("player must be SB or BB")

#### Tree Class

In [28]:
class Tree:
    def __init__(self, s, root, board):
        self.Stack = s
        self.nodes = [] #List of all nodes
        self.children = [0]
        self.parents = []
        self.board = board
    
    def getNumNodess(self):
        return len(self.decPts)
    
    def getStack():
        return self.Stack
    
    def addnode(self, node, parent):
        self.nodes.append(node)
        self.children.append([])
        if (parent == None):
            self.parents.append(None)
        else:
            parentsIndex = self.nodes.index(parent)
            self.children[parentsIndex].append(self.getNumPoints() - 1)
            self.parents.append(parentsIndex)
   

In [29]:
#Returns the equity of one hand vs another
def handVHandEquity(hand1, hand2, board):
    if conflicts(hand1,hand2) or conflicts(hand1,board) or conflicts(hand2,board):
        return -1
    else:
        res = pe.poker_eval(game = 'holdem', pockets = [hand1,hand1], board = board)
        total = res["info"][0]
        wins = res["eval"][0]["winhi"]
        ties = res["eval"][0]['tiehi']
        return (float(wins) + ties/2.0)/float(total)

## Solver

In [30]:
def updateRegrets(range1, range2, it):
    fraction = 1 - 1/(it + 2.0)
    for i in range(deckSize):
        for j in range(i + 1, deckSize):
            hand=[i,j]
            range1.setPercent(hand, (range1.getFreq(hand) * (fraction) + (range2.getFreq(hand)) * (1-fraction)))

In [31]:
def solveTree(tree, iterations):
    stack = tree.Stack
    board = tree.board
    equityObject = EquityObj(pe.string2card(board))
    sbFinalRange = Range()
    sbFinalRange.setAllFrequencies(0.5)
    bbFinalRange = Range()
    bbFinalRange.setAllFrequencies(0.5)
    
    
    for n in range(iterations):
        OptimalSBRange = Range()
        for i in range(deckSize):
            for j in range(i + 1, deckSize):
                hand = [i,j]
                bb_call_freq = bbFinalRange.getHandsNoConflicts(hand) / numP2Hands
                equity = handVsRange(hand, bbFinalRange, equityObject)
                evBet = (1 - bb_call_freq) * (stack + 1) + (bb_call_freq) * equity *2*stack
                evFold = stack - 0.5
                if (evBet > evFold):
                    OptimalSBRange.setPercent(hand,1)
                else:
                    OptimalSBRange.setPercent(hand, 0)
        updateRegrets(sbFinalRange, OptimalSBRange, n)
        OptimalBBRange = Range()
        for i in range(deckSize):
            for j in range(i + 1, deckSize):
                hand = [i,j]
                evFold = stack - 1
                evCall = 2 * stack * equity
                equity = handVsRange(hand, sbFinalRange, equityObject)

                if (evCall > evFold):
                    OptimalBBRange.setPercent(hand,1)
                else:
                    OptimalBBRange.setPercent(hand,0)
        
        updateRegrets(bbFinalRange, OptimalBBRange, n)
    print("Small Blind Range")
    display(sbFinalRange)
    print("Big Blind Range")
    display(bbFinalRange)
    return (sbFinalRange,bbFinalRange)

## Now We Will Run an Example Game

In [32]:
root = Node("SB", 20, 20,None)
tree = Tree(root.getStack("SB"),10,["__","__","__","__","__"])
result = solveTree(tree,100)

Small Blind Range


Big Blind Range


## Returns an exact frequency breakdown of each hand corresponding to an action
- In this case we only have two actions so the other action is simply 1 - (percentage) eg. call or fold, bet or check, call or raise

In [33]:
result[0].printRange()

('2h', '2h', 0.0)
('2h', '3h', 0.004950495049504954)
('2h', '4h', 0.004950495049504954)
('2h', '5h', 0.004950495049504954)
('2h', '6h', 0.004950495049504954)
('2h', '7h', 0.004950495049504954)
('2h', '8h', 0.004950495049504954)
('2h', '9h', 0.004950495049504954)
('2h', 'Th', 0.2227722772277228)
('2h', 'Jh', 0.9653465346534652)
('2h', 'Qh', 0.995049504950495)
('2h', 'Kh', 0.995049504950495)
('2h', 'Ah', 0.995049504950495)
('2h', '2d', 0.995049504950495)
('2h', '3d', 0.004950495049504954)
('2h', '4d', 0.004950495049504954)
('2h', '5d', 0.004950495049504954)
('2h', '6d', 0.004950495049504954)
('2h', '7d', 0.004950495049504954)
('2h', '8d', 0.004950495049504954)
('2h', '9d', 0.004950495049504954)
('2h', 'Td', 0.004950495049504954)
('2h', 'Jd', 0.004950495049504954)
('2h', 'Qd', 0.3613861386138615)
('2h', 'Kd', 0.995049504950495)
('2h', 'Ad', 0.995049504950495)
('2h', '2c', 0.995049504950495)
('2h', '3c', 0.004950495049504954)
('2h', '4c', 0.004950495049504954)
('2h', '5c', 0.00495049504950

('8d', 'Td', 0.995049504950495)
('8d', 'Jd', 0.995049504950495)
('8d', 'Qd', 0.995049504950495)
('8d', 'Kd', 0.995049504950495)
('8d', 'Ad', 0.995049504950495)
('8d', '2c', 0.004950495049504954)
('8d', '3c', 0.004950495049504954)
('8d', '4c', 0.004950495049504954)
('8d', '5c', 0.004950495049504954)
('8d', '6c', 0.004950495049504954)
('8d', '7c', 0.9059405940594061)
('8d', '8c', 0.995049504950495)
('8d', '9c', 0.995049504950495)
('8d', 'Tc', 0.995049504950495)
('8d', 'Jc', 0.995049504950495)
('8d', 'Qc', 0.995049504950495)
('8d', 'Kc', 0.995049504950495)
('8d', 'Ac', 0.995049504950495)
('8d', '2s', 0.004950495049504954)
('8d', '3s', 0.004950495049504954)
('8d', '4s', 0.004950495049504954)
('8d', '5s', 0.004950495049504954)
('8d', '6s', 0.004950495049504954)
('8d', '7s', 0.9059405940594061)
('8d', '8s', 0.995049504950495)
('8d', '9s', 0.995049504950495)
('8d', 'Ts', 0.995049504950495)
('8d', 'Js', 0.995049504950495)
('8d', 'Qs', 0.995049504950495)
('8d', 'Ks', 0.995049504950495)
('8d', '

('Ac', 'Jc', 0.0)
('Ac', 'Qc', 0.0)
('Ac', 'Kc', 0.0)
('Ac', 'Ac', 0.0)
('Ac', '2s', 0.995049504950495)
('Ac', '3s', 0.995049504950495)
('Ac', '4s', 0.995049504950495)
('Ac', '5s', 0.995049504950495)
('Ac', '6s', 0.995049504950495)
('Ac', '7s', 0.995049504950495)
('Ac', '8s', 0.995049504950495)
('Ac', '9s', 0.995049504950495)
('Ac', 'Ts', 0.995049504950495)
('Ac', 'Js', 0.995049504950495)
('Ac', 'Qs', 0.995049504950495)
('Ac', 'Ks', 0.995049504950495)
('Ac', 'As', 0.995049504950495)
('2s', '2h', 0.0)
('2s', '3h', 0.0)
('2s', '4h', 0.0)
('2s', '5h', 0.0)
('2s', '6h', 0.0)
('2s', '7h', 0.0)
('2s', '8h', 0.0)
('2s', '9h', 0.0)
('2s', 'Th', 0.0)
('2s', 'Jh', 0.0)
('2s', 'Qh', 0.0)
('2s', 'Kh', 0.0)
('2s', 'Ah', 0.0)
('2s', '2d', 0.0)
('2s', '3d', 0.0)
('2s', '4d', 0.0)
('2s', '5d', 0.0)
('2s', '6d', 0.0)
('2s', '7d', 0.0)
('2s', '8d', 0.0)
('2s', '9d', 0.0)
('2s', 'Td', 0.0)
('2s', 'Jd', 0.0)
('2s', 'Qd', 0.0)
('2s', 'Kd', 0.0)
('2s', 'Ad', 0.0)
('2s', '2c', 0.0)
('2s', '3c', 0.0)
('2s', '

In [34]:
result[1].printRange()

('2h', '2h', 0.0)
('2h', '3h', 0.995049504950495)
('2h', '4h', 0.004950495049504954)
('2h', '5h', 0.004950495049504954)
('2h', '6h', 0.004950495049504954)
('2h', '7h', 0.004950495049504954)
('2h', '8h', 0.004950495049504954)
('2h', '9h', 0.004950495049504954)
('2h', 'Th', 0.004950495049504954)
('2h', 'Jh', 0.004950495049504954)
('2h', 'Qh', 0.004950495049504954)
('2h', 'Kh', 0.004950495049504954)
('2h', 'Ah', 0.04455445544554454)
('2h', '2d', 0.995049504950495)
('2h', '3d', 0.995049504950495)
('2h', '4d', 0.004950495049504954)
('2h', '5d', 0.004950495049504954)
('2h', '6d', 0.004950495049504954)
('2h', '7d', 0.004950495049504954)
('2h', '8d', 0.004950495049504954)
('2h', '9d', 0.004950495049504954)
('2h', 'Td', 0.004950495049504954)
('2h', 'Jd', 0.004950495049504954)
('2h', 'Qd', 0.004950495049504954)
('2h', 'Kd', 0.004950495049504954)
('2h', 'Ad', 0.014851485148514854)
('2h', '2c', 0.995049504950495)
('2h', '3c', 0.995049504950495)
('2h', '4c', 0.004950495049504954)
('2h', '5c', 0.004

('9d', '5s', 0.004950495049504954)
('9d', '6s', 0.004950495049504954)
('9d', '7s', 0.004950495049504954)
('9d', '8s', 0.004950495049504954)
('9d', '9s', 0.004950495049504954)
('9d', 'Ts', 0.995049504950495)
('9d', 'Js', 0.004950495049504954)
('9d', 'Qs', 0.014851485148514854)
('9d', 'Ks', 0.9752475247524753)
('9d', 'As', 0.995049504950495)
('Td', '2h', 0.0)
('Td', '3h', 0.0)
('Td', '4h', 0.0)
('Td', '5h', 0.0)
('Td', '6h', 0.0)
('Td', '7h', 0.0)
('Td', '8h', 0.0)
('Td', '9h', 0.0)
('Td', 'Th', 0.0)
('Td', 'Jh', 0.0)
('Td', 'Qh', 0.0)
('Td', 'Kh', 0.0)
('Td', 'Ah', 0.0)
('Td', '2d', 0.0)
('Td', '3d', 0.0)
('Td', '4d', 0.0)
('Td', '5d', 0.0)
('Td', '6d', 0.0)
('Td', '7d', 0.0)
('Td', '8d', 0.0)
('Td', '9d', 0.0)
('Td', 'Td', 0.0)
('Td', 'Jd', 0.995049504950495)
('Td', 'Qd', 0.995049504950495)
('Td', 'Kd', 0.995049504950495)
('Td', 'Ad', 0.995049504950495)
('Td', '2c', 0.995049504950495)
('Td', '3c', 0.004950495049504954)
('Td', '4c', 0.004950495049504954)
('Td', '5c', 0.00495049504950495

('Kc', 'Tc', 0.0)
('Kc', 'Jc', 0.0)
('Kc', 'Qc', 0.0)
('Kc', 'Kc', 0.0)
('Kc', 'Ac', 0.995049504950495)
('Kc', '2s', 0.995049504950495)
('Kc', '3s', 0.014851485148514854)
('Kc', '4s', 0.014851485148514854)
('Kc', '5s', 0.02475247524752476)
('Kc', '6s', 0.03465346534653468)
('Kc', '7s', 0.0643564356435643)
('Kc', '8s', 0.995049504950495)
('Kc', '9s', 0.995049504950495)
('Kc', 'Ts', 0.995049504950495)
('Kc', 'Js', 0.995049504950495)
('Kc', 'Qs', 0.995049504950495)
('Kc', 'Ks', 0.995049504950495)
('Kc', 'As', 0.995049504950495)
('Ac', '2h', 0.0)
('Ac', '3h', 0.0)
('Ac', '4h', 0.0)
('Ac', '5h', 0.0)
('Ac', '6h', 0.0)
('Ac', '7h', 0.0)
('Ac', '8h', 0.0)
('Ac', '9h', 0.0)
('Ac', 'Th', 0.0)
('Ac', 'Jh', 0.0)
('Ac', 'Qh', 0.0)
('Ac', 'Kh', 0.0)
('Ac', 'Ah', 0.0)
('Ac', '2d', 0.0)
('Ac', '3d', 0.0)
('Ac', '4d', 0.0)
('Ac', '5d', 0.0)
('Ac', '6d', 0.0)
('Ac', '7d', 0.0)
('Ac', '8d', 0.0)
('Ac', '9d', 0.0)
('Ac', 'Td', 0.0)
('Ac', 'Jd', 0.0)
('Ac', 'Qd', 0.0)
('Ac', 'Kd', 0.0)
('Ac', 'Ad', 0.0)
(

('As', 'Th', 0.0)
('As', 'Jh', 0.0)
('As', 'Qh', 0.0)
('As', 'Kh', 0.0)
('As', 'Ah', 0.0)
('As', '2d', 0.0)
('As', '3d', 0.0)
('As', '4d', 0.0)
('As', '5d', 0.0)
('As', '6d', 0.0)
('As', '7d', 0.0)
('As', '8d', 0.0)
('As', '9d', 0.0)
('As', 'Td', 0.0)
('As', 'Jd', 0.0)
('As', 'Qd', 0.0)
('As', 'Kd', 0.0)
('As', 'Ad', 0.0)
('As', '2c', 0.0)
('As', '3c', 0.0)
('As', '4c', 0.0)
('As', '5c', 0.0)
('As', '6c', 0.0)
('As', '7c', 0.0)
('As', '8c', 0.0)
('As', '9c', 0.0)
('As', 'Tc', 0.0)
('As', 'Jc', 0.0)
('As', 'Qc', 0.0)
('As', 'Kc', 0.0)
('As', 'Ac', 0.0)
('As', '2s', 0.0)
('As', '3s', 0.0)
('As', '4s', 0.0)
('As', '5s', 0.0)
('As', '6s', 0.0)
('As', '7s', 0.0)
('As', '8s', 0.0)
('As', '9s', 0.0)
('As', 'Ts', 0.0)
('As', 'Js', 0.0)
('As', 'Qs', 0.0)
('As', 'Ks', 0.0)
('As', 'As', 0.0)
